### Requirements
To run this jupyter notebook yourself, first make sure that you have python v.3.11 installed on your system, together with the pip package manager.
As a first step you need to run `pip install -r requirements.txt`.

### Import
Next all necessary functions will be imported.
For this example the GPT-4 model will be used. Please enter your OpenAI key in the cell below.

In [ ]:
from io import BytesIO
from service.splitter import load_and_split_text
from service.knowledge_injection import inject_knowledge
from service.language_model_connection import KnowledgeLevel
from service.language_model_connection import LanguageModelConnection, LanguageModel
from service.language_model_connection import LanguageModelConnection, LanguageModel

llm = LanguageModelConnection(LanguageModel.GPT_4, "ENTER YOUR API KEY HERE")

### Determining the prior knowledge
As a help for the user, it is possible to use a questionnaire to determine the prior knowledge on regulation texts of a user. For this a questionnaire can be generated.

In [2]:
chunk = "1. This Regulation shall not invalidate any EU type-approvals granted to vehicles, systems, components or separate technical units which were granted in accordance with Regulation (EC) No <ent>78/2009</ent><ent_desc>This Regulation lays down requirements for the construction and functioning of motor vehicles and frontal protection systems in order to reduce the number and severity of injuries to pedestrians and other vulnerable road users who are hit by the fronts of vehicles and in order to avoid such collisions.</ent_desc>, Regulation (EC) No <ent>79/2009</ent><ent_desc>This Regulation establishes requirements for the type-approval of motor vehicles with regard to hydrogen propulsion and for the type-approval of hydrogen components and hydrogen systems. This Regulation also establishes requirements for the installation of such components and systems.</ent_desc> or Regulation (EC) No <ent>661/2009</ent><ent_desc>This Regulation establishes requirements: 1. for the type-approval of motor vehicles, their trailers and systems, components and separate technical units intended therefor with regard to their safety, 2. for the type-approval of motor vehicles, in respect of tyre pressure monitoring systems, with regard to their safety, fuel efficiency and CO2 emissions and, in respect of gear shift indicators, with regard to their fuel efficiency and CO2 emissions; and 3. for the type-approval of newly-manufactured tyres with regard to their safety, rolling resistance performance and rolling noise emissions.</ent_desc> and their implementing measures, by 5 July 2022, unless the relevant requirements applying to such vehicles, systems, components or separate technical units have been modified, or new requirements have been added, by this Regulation and the delegated acts adopted pursuant to it, as further specified in the implementing acts adopted pursuant to this Regulation."

print(llm.generate_questionnaire(chunk))

{'questions': [{'question': "What does 'EU type-approvals' refer to in the context of vehicle regulations?", 'answers': ['A. Certifications for food safety', 'B. Approvals for vehicle designs according to EU standards', 'C. Permissions for exporting vehicles outside the EU', 'D. Licenses for driving in the EU'], 'correct_answer': 'B. Approvals for vehicle designs according to EU standards'}, {'question': 'What is the primary purpose of implementing measures in vehicle regulations?', 'answers': ['A. To increase the cost of vehicle production', 'B. To standardize driving rules across the EU', 'C. To ensure compliance with updated safety and efficiency standards', 'D. To restrict the import of non-EU vehicles'], 'correct:answer': 'C. To ensure compliance with updated safety and efficiency standards'}, {'question': "What does the term 'delegated acts' imply in the context of EU regulations?", 'answers': ['A. Acts performed by temporary delegates in the EU parliament', 'B. Specific powers g

### Chunk Summary
As an intermediate step in the summarization process, a summary for each chunk is generated. This chunk summary consists of the stakeholders involved, all important information of the chunk and a complete summary of the chunk.

In [3]:
print(llm.generate_chunk_summary(chunk))

{'stakeholder': ['vehicle manufacturers', 'system and component manufacturers'], 'key_information': ['Existing EU type-approvals for vehicles and related components remain valid unless modified by new requirements.', 'Regulation (EC) No 78/2009 focuses on reducing injuries to pedestrians from vehicle fronts.', 'Regulation (EC) No 79/2009 deals with hydrogen propulsion systems in vehicles.', 'Regulation (EC) No 661/2009 covers safety, fuel efficiency, and emissions standards for vehicles and their components.', 'Changes or additions to requirements by new regulations or delegated acts may invalidate existing approvals after 5 July 2022.'], 'chunk_summary': 'Existing EU type-approvals for vehicles, systems, and components will continue to be valid unless there are modifications or additions to the requirements by new regulations or delegated acts. These include standards for pedestrian safety, hydrogen propulsion systems, and vehicle safety and emissions. Vehicle and system manufacturers

### Full Summary
As a last step the full workflow is tested. Here, a file is split into chunks, knowledge gets injected, and a final summary is generated. In this example a user without prior knowledge is used. You can change the XML file, by putting it into the tmp folder and changing the file path.

In [4]:
# Change file location here
with open("tmp/CELEX_02019R2144-20220905_EN_TXT.pdf.tei.xml", "rb") as fh:
    buf = BytesIO(fh.read())

# split the text into chunks
docs = load_and_split_text(text=buf.getvalue(), chunk_size=15000, chunk_overlap=0, splitter_type="Text Structure")
summarys = []

# inject knowledge and generate chunk summaries
for i,doc in enumerate(docs):
    doc = inject_knowledge(doc)
    summarys.append(llm.generate_chunk_summary(doc))

# generate full summary for no prior knowledge
# available knowledge levels are: NO, BASIC and EXPERT
summary = llm.generate_policy_summary(summarys, knowledge_level=KnowledgeLevel.NO.name)
display(summary)

"The regulation mandates type-approval for new vehicles and components, focusing on safety, environmental performance, and advanced vehicle systems. It specifies detailed requirements for systems like tyre pressure monitoring and intelligent speed assistance. Vehicle manufacturers must ensure compliance with these requirements to enhance the safety of vehicle occupants and vulnerable road users. Additionally, the regulation outlines the necessity for manufacturers to minimize injury risks and adhere to technical requirements. It also introduces mandatory advanced vehicle systems such as event data recorders and intelligent speed assistance for all motor vehicles. Furthermore, implementing acts will establish uniform procedures and technical specifications for type-approval to ensure consistency and reliability in vehicle safety standards.The regulation mandates type-approval for new vehicles and components, focusing on safety, environmental performance, and advanced vehicle systems. It